# 0. Import Packages

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time
import re
import pandas as pd
from datetime import datetime
from datetime import date
from datetime import date, timedelta
import random
import threading
import multiprocessing as np
from multiprocessing import Pool
import concurrent.futures

# 1. Functions

## 1.1. Loop Through Dates

In [2]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

In [12]:
driver = webdriver.Chrome()
driver.get("https://www.politico.com/search/20?adv=true&start=02/01/2016&end=02/01/2016")
#print(driver.find_element(By.CSS_SELECTOR, "a.button").text)
print(driver.find_element(By.CSS_SELECTOR, "a.button:nth-child(2)").text)

Next page »


# 2. Scraping News Media

## 2.1. Wall Street Journal

### 2.1.1. Function - Get URLS

In [3]:
def getWallStreetJournalURLS(start_date, end_date, username, password):
    #put dates in date format
    start_date = datetime.strptime(start_date, "%d/%m/%Y")
    end_date = datetime.strptime(end_date, "%d/%m/%Y")

    #Define list of variables to store 
    WSJ_date = []
    WSJ_link = []

    #Go to the website
    fireFoxOptions = webdriver.FirefoxOptions()
    fireFoxOptions.add_argument('--ignore-certificate-errors')
    fireFoxOptions.add_argument('--allow-running-insecure-content')
    fireFoxOptions.add_argument("--headless")
    fireFoxOptions.add_argument("--disable-gpu")
    fireFoxOptions.add_argument("--no-sandbox")
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    fireFoxOptions.add_argument(f'user-agent={user_agent}')
    driver_WSJ = webdriver.Firefox(options=fireFoxOptions)
    #driver_WSJ = webdriver.Firefox()
    driver_WSJ.get("https://sso.accounts.dowjones.com/login-page?op=localop&scope=openid%20idp_id%20roles%20email%20given_name%20family_name%20djid%20djUsername%20djStatus%20trackid%20tags%20prts%20suuid%20createTimestamp&client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&response_type=code&redirect_uri=https%3A%2F%2Faccounts.wsj.com%2Fauth%2Fsso%2Flogin&nonce=ab6a473a-cfa6-4714-8fad-b6dff98f5f18&ui_locales=en-us-x-wsj-223-2&mars=-1&ns=prod%2Faccounts-wsj&state=8rChOTDzC_Y_AK-i.TJAixN_XjsWxwUEEPoHg2OPCaX6qRBu4nGSk5fqLliY4H0B5F7gj_57-XH-YBWGS&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin")

    #put in user_name:
    time.sleep(3) #makes sure field is fully loaded
    username_field = driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[1]/div[2]/input")
    username_field.send_keys(username)

    #continue to password:
    driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[6]/div[1]/button[2]").click()

    #input password:
    time.sleep(3) #makes sure field is fully loaded
    password_field = driver_WSJ.find_element(By.XPATH, "//*[@id='password-login-password']")
    password_field.send_keys(password)

    #click on sign in:
    driver_WSJ.find_element(By.XPATH, "//*[@id='password-login']/div/form/div/div[5]/div[1]/button").click()

    try:
        #accept cookies
        time.sleep(10)
        driver_WSJ.switch_to.frame(WebDriverWait(driver_WSJ,30).until(EC.presence_of_element_located((By.ID, 'sp_message_iframe_718122'))))
        driver_WSJ.find_element(By.CSS_SELECTOR, "button.message-component:nth-child(2)").click()
        driver_WSJ.switch_to.default_content() 
        time.sleep(3)
    except:
        pass

    scrape_time = time.time() #starting the timing

    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%d/%m/%Y")
        print(date)
        year = "%02d" % (int(date.split('/')[2]),)
        month = "%02d" % (int(date.split('/')[1]),)
        day = "%02d" % (int(date.split('/')[0]),)
        page = "%02d" % (1,) #we assume there is always at least one page
        url = f"https://www.wsj.com/news/archive/{year}/{month}/{day}?page={page}"
        test = 0
        while(test < 100):
            try:
                driver_WSJ.get(url) #go to the page in the WSJ archive for the given date
                break
            except:
                test += 1
                time.sleep(10)

        #check if there are multiple pages and if so, visit these as well
        pages_load = 0
        while(pages_load < 100):
            try:
                pages = WebDriverWait(driver_WSJ, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "WSJTheme--pagepicker-total--Kl350I1l ")))
                page_total = re.findall(r'\d+', pages.text)
                nr_pages = int(page_total[0])
                break
            except:
                pages_load += 1
                driver_WSJ.get(url)

        for p in range(1,nr_pages+1):
            page = "%02d" % (p,)
            url = f"https://www.wsj.com/news/archive/{year}/{month}/{day}?page={page}"
            urls_load = 0
            while(urls_load < 100):
                try:
                    driver_WSJ.get(url) #go to the page in the WSJ archive for the given date
                    break
                except:
                    driver_WSJ.get(url)

            #get the article urls for the articles on the other pages, if there are more than 1
            articles = WebDriverWait(driver_WSJ,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
            for a in range(0,len(articles)):
                WSJ_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
                WSJ_date.append(date)
    
    driver_WSJ.quit()
    
    WSJ = ["Wall_Street_Journal"] * len(WSJ_link)
    data = {"Date" : WSJ_date, "News Paper" : WSJ, "Link" : WSJ_link}
    Wall_Street_Journal = pd.DataFrame(data)

    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/URLS/Wall_Street_Journal_{start_date}_{end_date}_URLS"
    Wall_Street_Journal.to_parquet(path) 
    
    print("--- %s seconds ---" % (time.time() - scrape_time))

### 2.1.2. Function - Articles

In [4]:
def getWallStreetJournalArticles(username, password, dataframe):
    WSJ_article_content = []
    WSJ_title = []

    #FIREFOX
    #Declare the driver and go to website
    #fireFoxOptions = webdriver.FirefoxOptions()
    #fireFoxOptions.add_argument('--ignore-certificate-errors')
    #fireFoxOptions.add_argument('--allow-running-insecure-content')
    #fireFoxOptions.add_argument("--headless")
    #fireFoxOptions.add_argument("--disable-gpu")
    #fireFoxOptions.add_argument("--no-sandbox")
    #user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    #fireFoxOptions.add_argument(f'user-agent={user_agent}')
    #driver_WSJ = webdriver.Firefox(options=fireFoxOptions)
    
    #CHROME
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox") # linux only
    chrome_options.add_argument("--headless")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver_WSJ = webdriver.Chrome(options=chrome_options)
   # driver_WSJ = webdriver.Chrome()
    driver_WSJ.get("https://sso.accounts.dowjones.com/login-page?op=localop&scope=openid%20idp_id%20roles%20email%20given_name%20family_name%20djid%20djUsername%20djStatus%20trackid%20tags%20prts%20suuid%20updated_at&client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&response_type=code&redirect_uri=https%3A%2F%2Faccounts.wsj.com%2Fauth%2Fsso%2Flogin&nonce=5e93fe71-314a-44b9-bb2d-4a0e5a3167a4&ui_locales=en-us-x-wsj-223-2&mars=-1&ns=prod%2Faccounts-wsj&state=zJgHWictetlLHLG6.Uob1DvxnFA4kIkiqJAY80zp2N4FxS3WdBvV-VuTFdpM&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin")

    #put in user_name:
    time.sleep(3) #makes sure field is fully loaded
    username_field = driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[1]/div[2]/input")
    username_field.send_keys(username)

    #continue to password:
    driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[6]/div[1]/button[2]").click()

    #input password:
    time.sleep(3) #makes sure field is fully loaded
    password_field = driver_WSJ.find_element(By.XPATH, "//*[@id='password-login-password']")
    password_field.send_keys(password)

    #click on sign in:
    driver_WSJ.find_element(By.XPATH, "//*[@id='password-login']/div/form/div/div[5]/div[1]/button").click()

    try:
        #accept cookies
        time.sleep(20)
        driver_WSJ.switch_to.default_content()
        driver_WSJ.switch_to.frame(WebDriverWait(driver_WSJ,30).until(EC.presence_of_element_located((By.ID, 'sp_message_iframe_718122'))))
        driver_WSJ.find_element(By.CSS_SELECTOR, "button.message-component:nth-child(2)").click()
        print("accepted")
        time.sleep(3)
    except:
        pass

    scrape_time = time.time()
    #go to all the articles and scrape the content
    links  = list(dataframe["Link"])
    for u in range(0, len(links)):
        trys = 0
        while(trys < 100):
            try:
                driver_WSJ.get(links[u])
                break
            except:
                trys += 1
                time.sleep(10)

        try:
            #extract the content and add to a variables
            content = WebDriverWait(driver_WSJ, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'article-content  ')))
            text = content.find_elements(By.TAG_NAME, 'p')
            article_text = ""
            for t in range(0,len(text)):
                article_text += " " + text[t].text

            WSJ_article_content.append(article_text) #add text to the list
            #WSJ_article_content.append(content.text) #add text to the list

        except: #we don't have access to the article
            WSJ_article_content.append("NO ACCESS")
            pass #go back to page with all articles

        try:
            #collect the title as well
            title = WebDriverWait(driver_WSJ,10).until(EC.presence_of_element_located((By.CLASS_NAME, "wsj-article-headline"))).text
            WSJ_title.append(title)
        except:
            try:
                #there are 2 main formats in which the titles are present in the HTML
                title = WebDriverWait(driver_WSJ,10).until(EC.presence_of_element_located((By.CLASS_NAME, "bigTop__hed"))).text
                WSJ_title.append(title)

            except:
                #add error if there is some unexpected layout -> this way our arrays will be of the same length and we will be able
                #to construct a dataframe in the end
                WSJ_title.append("ERROR")
                pass

    driver_WSJ.quit()

    WSJ = ["Wall_Street_Journal"] * len(WSJ_article_content)
    data = {"Title" : WSJ_title, "Text" : WSJ_article_content, "Link" : dataframe["Link"]}
    Wall_Street_Journal = pd.DataFrame(data)
    Wall_Street_Journal = pd.merge(Wall_Street_Journal, dataframe, on = "Link")
    return(Wall_Street_Journal)

    print("--- %s seconds ---" % (time.time() - scrape_time))

## 2.2. Washington Post

### 2.2.1. Function - Get URLS

In [5]:
def getWashingtonPostURLS(start_date, end_date, username, password):
    #put dates in date format
    start_date = datetime.strptime(start_date, "%d/%m/%Y")   
    end_date = datetime.strptime(end_date, "%d/%m/%Y")


    #put them in correct format


    #Declare all list variables for the output
    WP_date = []
    WP_link = []
    
    #FIREFOX
    #Declare the driver and go to website
    #fireFoxOptions = webdriver.FirefoxOptions()
    #fireFoxOptions.add_argument('--ignore-certificate-errors')
    #fireFoxOptions.add_argument('--allow-running-insecure-content')
    #fireFoxOptions.add_argument("--headless")
    #fireFoxOptions.add_argument("--disable-gpu")
    #fireFoxOptions.add_argument("--no-sandbox")
    #user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    #fireFoxOptions.add_argument(f'user-agent={user_agent}')
    #driver = webdriver.Firefox(options=fireFoxOptions)
    
    #CHROME
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox") # linux only
    chrome_options.add_argument("--headless")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    driver = webdriver.Chrome()
    # Initialize the driver


    #Go the the searchpage of the Washington Post
    driver.get("https://www.washingtonpost.com/search/?query=+&facets=%7B%22time%22%3A%22all%22%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%5D%2C%22author%22%3A%5B%5D%7D")
    #Accept Cookies
    try:
        driver.save_screenshot("cookies.png")
        WebDriverWait(driver,20).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))).click()
    except:
        pass
    
    #Log-in into Washington Post account
    try: #it is possible that account is already logged in! 
        #Sign in into Washington Post account
        #Click on Sign In
        #WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/div[1]/nav/div[4]/div[2]/a/p"))).click()
        driver.get("https://www.washingtonpost.com/subscribe/signin/?next_url=https%3A%2F%2Fwww.washingtonpost.com&nid=top_pb_signin&arcId=&itid=nav_sign_in")
        time.sleep(5)
        driver.save_screenshot("Login.png")
        #located username field
        time.sleep(2) #field needs to completely load
        username_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='username']")))
        username_field.send_keys(username) #send username
        username_field.send_keys(Keys.RETURN) #press enter

        #Located password field
        time.sleep(2) #field needs to completely load
        password_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='password']")))
        password_field.send_keys(password)
        password_field.send_keys(Keys.ENTER)

        #Go back to the search page
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "wpds-c-jlBemH "))).click()
        go_to_search = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "query")))
        go_to_search.send_keys(search_term)
        go_to_search.send_keys(Keys.ENTER)

    except:
        #you already logged-in -> continue 
        pass
    
    search_page_load = 0
    while(search_page_load < 100):
        try:
            driver.get("https://www.washingtonpost.com/search/?query=+&facets=%7B%22time%22%3A%22all%22%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%5D%2C%22author%22%3A%5B%5D%7D")
            break
        except:
            search_page_load += 1
            
    scrape_time = time.time()
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%m/%d/%y")

        #Locate the searchbar, send searchterm and press enter
        
        time.sleep(10)
        driver.save_screenshot("Searchbar.png")
        searchbar = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "aa-Input")))
        searchbar.send_keys(" ")
        searchbar.send_keys(Keys.ENTER)

        #Select time period
        #select periode specification
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/button/div/div[2]/span"))).click()

        #select custom time
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[1]/button/div/div[2]"))).click()
        time.sleep(1)

        #Send start date
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/div[1]/div/div/div[1]/input"))).send_keys(date)

        #Empty automatic end date
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/div[2]/div/div/div[1]/input"))).clear()

        #Send end date
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/div[2]/div/div/div[1]/input"))).send_keys(date)

        #Press Apply
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/button[2]"))).click()
        time.sleep(5)
        driver.refresh()
        #click to load more on the page untill no longer possible
        loading = True
        while(loading):
            try:
                driver.save_screenshot("loading.png")
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[3]/button"))).click()
            except:
                loading = False
        
        driver.save_screenshot("articles.png")
        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))

        for a in range(0,len(articles)):
            WP_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            WP_date.append(single_date.strftime("%d/%m/%Y"))
    
    driver.quit()
    
    WP = ["Washington_Post"] * len(WP_link)
    data = {"Date" : WP_date, "News Paper" : WP, "Link" : WP_link}
    WashingtonPost = pd.DataFrame(data)

    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/URLS/Washington_Post_{start_date}_{end_date}_URLS"
    #WashingtonPost.to_parquet(path) 
    return(WashingtonPost)

    print("--- %s seconds ---" % (time.time() - scrape_time))

In [8]:
URLS_WP = getWashingtonPostURLS("01/01/2016", "01/01/2016", "tb.boedttibo@gmail.com", "ThesisR&T")

In [49]:
len(URLS_WP)

48

### 2.2.2. Function - Articles

In [57]:
def getWashingtonPostArticles(username, password, dataframe):
    WP_article_content = []
    WP_title = []

    #FIREFOX
    #Declare the driver and go to website
    #fireFoxOptions = webdriver.FirefoxOptions()
    #fireFoxOptions.add_argument('--ignore-certificate-errors')
    #fireFoxOptions.add_argument('--allow-running-insecure-content')
    #fireFoxOptions.add_argument("--headless")
    #fireFoxOptions.add_argument("--disable-gpu")
    #fireFoxOptions.add_argument("--no-sandbox")
    #user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    #fireFoxOptions.add_argument(f'user-agent={user_agent}')
    #driver_WSJ = webdriver.Firefox(options=fireFoxOptions)

    #CHROME
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox") # linux only
    chrome_options.add_argument("--headless")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    #driver = webdriver.Chrome()

    #Go the the searchpage of the Washington Post
    driver.get("https://www.washingtonpost.com/search/?query=+&facets=%7B%22time%22%3A%22all%22%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%5D%2C%22author%22%3A%5B%5D%7D")

    try:
        #Accept Cookies
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))).click()
    except:
        print("no cookies")
        pass
    #Log-in into Washington Post account
    try: #it is possible that account is already logged in! 

        #Sign in into Washington Post account
        #Click on Sign In
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/div[1]/nav/div[4]/div[2]/a/p"))).click()

        #located username field
        time.sleep(2) #field needs to completely load
        username_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='username']")))
        username_field.send_keys(username) #send username
        username_field.send_keys(Keys.RETURN) #press enter

        #Located password field
        time.sleep(2) #field needs to completely load
        password_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='password']")))
        password_field.send_keys(password)
        password_field.send_keys(Keys.ENTER)

        time.sleep(10)

    except:
        #you already logged-in -> continue 
        pass

    scrape_time = time.time()
    links = list(dataframe["Link"])
    for u in range(0, len(links)):
            trys = 0
            while(trys < 100):
                try:
                    #go to every article
                    driver.get(links[u])
                    break
                except:
                    trys += 1

            text = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.TAG_NAME, "article"))).text
            try:
                text = text[text.index("Share")+len("Share")+1:text.rindex("Gift Article")]
                WP_article_content.append(text)
            except:
                try:
                    text = text[text.index("Share")+len("Share")+1:text.rindex("Comments")]
                    WP_article_content.append(text)
                except:
                    try:
                        text = text[text.index("tb.boedttibo")+len("tb.boedttibo")+1:text.rindex("Comments")]
                        WP_article_content.append(text)
                    except:
                        WP_article_content.append(text)
                        #WP_article_content.append("ERROR")
                        pass
            #idea is: We take the full text and before we have the article content the last pice of text is "share" and immediatly after the content
            #we have "comments" this way we are able to only extract the article content
            #the reasons for this is that each article has a more or less different html dependent on their "type"

            #there are different formats of titles, in this order all titles are located
            try:
                title = driver.find_element(By.CSS_SELECTOR, "#main-content > span:nth-child(2)").text
                WP_title.append(title)
            except:
                try:
                    title = driver.find_element(By.CSS_SELECTOR, "#main-content").text
                    WP_title.append(title)
                except:
                    try:
                        title = driver.find_element(By.CSS_SELECTOR, "#main-content > span").text
                        WP_title.append(title)
                    except:
                        WP_title.append("ERROR")
                        pass

    #Close driver
    driver.quit()

    #How long did the scraping took? 
    print("--- %s seconds ---" % (time.time() - scrape_time))

    #Create output file
    #Make the dataframe
    WP = ["Washington Post"] * len(WP_article_content)
    data = {"Title" : WP_title, "Text" : WP_article_content, "Date" : dataframe["Date"], "News Paper" : dataframe["News Paper"], "Link" : dataframe["Link"]}
    WashingtonPost = pd.DataFrame(data)
    return(WashingtonPost)

In [62]:
getWashingtonPostArticles("tb.boedttibo@gmail.com", "ThesisR&T", URLS_WP[0:20])

--- 64.20283389091492 seconds ---


,Title,Text,Date,News Paper,Link
0,Here’s another sign the era of mass incarcerat...,Keith Humphreys is a professor of psychiatry a...,01/01/2016,Washington_Post,https://www.washingtonpost.com/news/wonk/wp/20...
1,Record flooding in the U.K. is just the latest...,December witnessed a spate of extreme global w...,01/01/2016,Washington_Post,https://www.washingtonpost.com/news/energy-env...
2,Will Compton is rubbing elbows with his heroes...,Will Compton admires DeAngelo Hall. Thanks to ...,01/01/2016,Washington_Post,https://www.washingtonpost.com/news/football-i...
3,It’s high season for online dating — plot your...,"If dating is a game, online dating is a game o...",01/01/2016,Washington_Post,https://www.washingtonpost.com/news/soloish/wp...
4,The Fix’s 2016 predictions!,In these final days of 2015 -- the most unpred...,01/01/2016,Washington_Post,https://www.washingtonpost.com/news/the-fix/wp...
5,What UFC 195 lacks in personality it could mak...,UFC will kick off a new year of pay-per-views ...,01/01/2016,Washington_Post,https://www.washingtonpost.com/news/early-lead...
6,Want to do something good for your health? Try...,"Every day, we are confronted with choices abou...",01/01/2016,Washington_Post,https://www.washingtonpost.com/posteverything/...
7,"Kanye West calls out Nike, mentions LeBron Jam...",Updated post: Michael Jordan’s son responds to...,01/01/2016,Washington_Post,https://www.washingtonpost.com/news/early-lead...
8,"NBA\n7 p.m. Orlando at Washington » CSN, WNEW ...","NBA\n7 p.m. Orlando at Washington » CSN, WNEW ...",01/01/2016,Washington_Post,https://www.washingtonpost.com/sports/tv-and-r...
9,Derrick Henry rushes for 75 yards and two touc...,"ARLINGTON, Texas — Porous on the first night o...",01/01/2016,Washington_Post,https://www.washingtonpost.com/sports/colleges...


In [59]:
tijd = time.time()

import concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    # Submit the two instances of the scraping function to the executor
    future1 = executor.submit(getWashingtonPostArticles, "tb.boedttibo@gmail.com", "ThesisR&T", URLS_WP[0:5])
    future2 = executor.submit(getWashingtonPostArticles, "tb.boedttibo@gmail.com", "ThesisR&T", URLS_WP[5:10])
    future3 = executor.submit(getWashingtonPostArticles, "tb.boedttibo@gmail.com", "ThesisR&T", URLS_WP[10:15])
    future4 = executor.submit(getWashingtonPostArticles, "tb.boedttibo@gmail.com", "ThesisR&T", URLS_WP[15:20])
    
    # Wait for the scraping functions to complete
    concurrent.futures.wait([future1, future2, future3, future4])
    
print("--------------------------")   
print("--- %s seconds ---" % (time.time() - tijd))

no cookies
no cookies
--- 26.277382135391235 seconds ---
--- 35.9525420665741 seconds ---
--- 39.549072265625 seconds ---
--- 39.95678997039795 seconds ---
--------------------------
--- 71.04116654396057 seconds ---


## 2.3. New York Times

### 2.3.1. Function - Get URLS

In [4]:
def getNewYorkTimesURLS(start_date, end_date, username, password):
    #put dates in date format
    start_date = datetime.strptime(start_date, "%d/%m/%Y") 
    end_date = datetime.strptime(end_date, "%d/%m/%Y")

    #Declare all list variables for the output
    NYT_date = []
    NYT_link = []

    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox") # linux only
    chrome_options.add_argument("--headless")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://myaccount.nytimes.com/auth/login?response_type=cookie&client_id=vi&redirect_uri=https%3A%2F%2Fwww.nytimes.com%2Fsubscription%2Fonboarding-offer%3FcampaignId%3D7JFJX%26EXIT_URI%3Dhttps%253A%252F%252Fwww.nytimes.com%252F&asset=masthead")
    time.sleep(5)
    driver.save_screenshot("test.png")
    #not a robot
    input("Press Enter to continue...")

    #Select log-in field
    login = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "email")))

    #Send username
    login.send_keys(username)

    #Press continue
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#myAccountAuth > div > div > div > form > div > div.css-bho3kg-buttonWrapper-buttonStyles-Button > button"))).click()

    #Select password field
    password_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "password")))

    #send password
    password_field.send_keys(password)

    #Press continue
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#myAccountAuth > div > div > form > div > div.css-1nkv26b-buttonWrapper-buttonStyles-Button > button"))).click()

    time.sleep(1)

    #go to search page
    search_page_loaded = 0
    while(search_page_loaded < 100):
        try:
            driver.get("https://www.nytimes.com/search?dropmab=false&query=&sort=best")
            break
        except:
            search_page_loaded += 1

    #Accept Cookies
    try:
        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CLASS_NAME, "banner__container__cta--accept"))).click()
    except:
        pass

    scrape_time = time.time()
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%Y%m%d")

        url = f"https://www.nytimes.com/search?dropmab=false&endDate={date}&query=%20&sort=best&startDate={date}"
        url_loaded = 0
        while(url_loaded < 100):
            try:
                driver.get(url)
                break
            except:
                url_loaded += 1

        #click to load more on the page untill no longer possible
        loading = True
        while(loading):
            try:
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='site-content']/div/div[2]/div[2]/div/button"))).click()
            except:
                loading = False

        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-1l4w6pd")))
        dates = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-17ubb9w")))

        date_list = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-17ubb9w")))
        true_date = date_list[0].text
        for a in range(0, len(articles)):
            if(true_date == date_list[a].text):
                NYT_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
                NYT_date.append(single_date.strftime("%d/%m/%Y"))
    
    driver.quit()
    
    NYT = ["New_York_Times"] * len(NYT_link)
    data = {"Date" : NYT_date, "News Paper" : NYT, "Link" : NYT_link}
    New_York_Times = pd.DataFrame(data)
    
    print("--- %s seconds ---" % (time.time() - scrape_time))
    
    return(New_York_Times)
    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/URLS/New_York_Times_{start_date}_{end_date}_URLS"
    New_York_Times.to_parquet(path)

## 2.4. Politico

### 2.4.1. Function - Get URLS

In [ ]:
def getPolitico(start_date, end_date):
    #put dates in date format
    end_date = datetime.strptime(end_date, "%d/%m/%Y")
    start_date = datetime.strptime(start_date, "%d/%m/%Y")

    #Declare all list variables for the output
    P_date = []
    P_link = []

    fireFoxOptions = webdriver.FirefoxOptions()
    fireFoxOptions.add_argument('--ignore-certificate-errors')
    fireFoxOptions.add_argument('--allow-running-insecure-content')
    fireFoxOptions.add_argument("--headless")
    fireFoxOptions.add_argument("--disable-gpu")
    fireFoxOptions.add_argument("--no-sandbox")
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    fireFoxOptions.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Firefox(options=fireFoxOptions)

    #go to the website
    driver.get("https://www.politico.com/search?adv=true")

    #accept cookies
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))).click()
    except:
        pass

    scrape_time = time.time()
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%m/%d/%Y")
        test = 0
        while(test < 100):
            try:
                driver.get(f"https://www.politico.com/search/1?adv=true&start={date}&end={date}")
                break
            except:
                test += 1
                time.sleep(10)
        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
        for a in range(0, len(articles)):
            P_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            P_date.append(date)

        #get the number of pages to know how much you should click .pagination > ol:nth-child(2) > li:nth-child(3) > a:nth-child(1)
        try:
            nr_pages = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pagination > ol:nth-child(2) > li:nth-child(6) > a:nth-child(1)"))).text
        except:
            nr_pages = 1
            pass

        for p in range(1, int(nr_pages)):
            url = f"https://www.politico.com/search/{p+1}?adv=true&start={date}&end={date}"
            driver.get(url)

        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
        for a in range(0, len(articles)):
            P_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            P_date.append(single_date.strftime("%d/%m/%Y"))

    driver.quit()
    print("--- %s seconds ---" % (time.time() - scrape_time))

    P = ["Politico"] * len(P_link)
    data = {"Date" : P_date, "News Paper" : P, "Link" : P_link}
    Politico = pd.DataFrame(data)

    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/URLS/Politico_{start_date}_{end_date}_URLS"
    Politico.to_parquet(path)

# 3. Running Functions

## 3.1. Wall Street Journal

## 3.2. Washington Post

In [ ]:
#getWashingtonPostURLS("01/01/2016", "01/01/2016", "tb.boedttibo@gmail.com", "ThesisR&T")

In [ ]:
WP_URLS = pd.read_parquet("Data/URLS/Wall_Street_Journal_01012016_01012016_URLS")

In [ ]:
WP_URLS

## 3.3. New York Times

## 3.4. Politico